In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install transformers[sentencepiece]
! pip install tokenizers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 404 kB/s 
     |████████████████████████████████| 895 kB 39.5 MB/s 
     |████████████████████████████████| 3.3 MB 47.9 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 1.2 MB 52.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import json
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import transformers
import collections
from transformers import TFAlbertModel, AlbertTokenizerFast, AlbertTokenizer


In [ ]:
class DataElement:
    def __init__(self, question, context, answer, answer_start, answer_end):
        self.question = question
        self.context = context
        self.answer = answer
        self.answer_start = answer_start
        self.answer_end = answer_end
        
    def preprocess(self):
        # create context vector with answers marked
        context_vector = [0] * len(self.context)
        for index in range(self.answer_start, self.answer_end):
            context_vector[index] = 1
            
        # tokenize context   
        tokenized_context = tokenizer(self.context, return_offsets_mapping=True)
        context_offsets = tokenized_context['offset_mapping']

        # find answer token indices 
        answer_token_index = []
        for index, (start, end)  in enumerate(context_offsets):
            if sum(context_vector[start:end]) > 0: # if token is answer
                answer_token_index.append(index)
        
        if len(answer_token_index) == 0:
            return 0
        
        # start and end token index
        start_token_index = answer_token_index[0]
        end_token_index = answer_token_index[-1]
        
        # tokenize question
        tokenized_question = tokenizer(self.question, return_special_tokens_mask=True)

        # create inputs       
        input_ids = tokenized_context['input_ids'] + tokenized_question['input_ids'][1:]
        token_type_ids = [0] * len(tokenized_context['input_ids']) + [1] * len(tokenized_question['input_ids'][1:])
            
        attention_mask = [1] * len(input_ids)
        
        # padding for equal lenght sequence
        padding_length = max_len - len(input_ids)
        if padding_length > 0: # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length) # len(input) [1] + padding [0]
            token_type_ids = token_type_ids + ([0] * padding_length) # context [0] + question [1] + padding [0]
        elif padding_length < 0:
            return 0
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_index = start_token_index
        self.end_token_index = end_token_index
        self.context_token_to_char = context_offsets
        return 1
      
    def class_print(self):
        print("Question: {}\nAnswer: {}\nAnswer Start: {}\nAnswer End: {}\nContext: {}".format(self.question, 
                                                                                              self.answer,  
                                                                                              self.answer_start, 
                                                                                              self.answer_end,
                                                                                              self.context))

In [ ]:
def read_json(file_name):
    with open(file_name, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
    return data

def json_to_list(json_dataset):
    dataset = []
    for paragraph_element in json_dataset["data"]:
        for question_element in paragraph_element["qas"]:
            dataset.append(DataElement(question_element["question"],
                                       paragraph_element["text"],
                                       question_element["answer"],
                                       question_element["answer_start"],
                                       question_element["answer_end"]))
    print("Number of questions: ", len(dataset))
    return dataset

def create_input_targets(dataset):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_index": [],
        "end_token_index": [],
    }
    i=0
    for item in dataset:
        # print(i)
        i = i + 1
        # print(item.class_print())
        for key in dataset_dict:
            dataset_dict[key].append(getattr(item, key))
            
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
        
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    
    y = [dataset_dict["start_token_index"], dataset_dict["end_token_index"]]
    return x, y

def find_max_length(dataset):
    max_ = 0
    index = 0
    i = 0
    for element in dataset:
        tokenized_question = tokenizer.encode(element.question)
        tokenized_context = tokenizer.encode(element.context)
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        
        if len(input_ids) > max_:
            max_ = len(input_ids)
            index = i
        i += 1
        
    print("Max length: {}, Index: {}".format(max_, index))
    return max_

def train_test_split(dataset):
    random.shuffle(dataset) 
    cut = int(len(dataset)*0.1)
    train, test = dataset[:-cut], dataset[-cut:] 
    
    return train, test

def create_model():
    ## ALBERT encoder MODEL_NAME is defined previously 
    encoder = TFAlbertModel.from_pretrained(MODEL_NAME)
    encoder.save_pretrained(save_path+"/")

    # QA model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder.albert(input_ids, 
                             token_type_ids=token_type_ids, 
                             attention_mask=attention_mask)[0]
    
    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)
    
    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)
    
    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)
    
    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model

In [ ]:
import pickle

def save_data_as_file(data, file_name):
  with open(path + file_name + ".dat", "wb") as f:
    pickle.dump(data, f)

def read_saved_data(file_name):
  with open(path + file_name + ".dat", "rb") as f:
    data = pickle.load(f)
    return data

In [ ]:
path = "/content/gdrive/MyDrive/Q&A projesi/"
models_path = path + "models/"
MODEL_NAME = "loodos/albert-base-turkish-uncased"
save_path = models_path+"loodos-albert-base-turkish-uncased"


In [ ]:
# slow_tokenizer = AlbertTokenizerFast.from_pretrained(MODEL_NAME, do_lower_case=False, keep_accents=True)


# if not os.path.exists(save_path):
#     os.makedirs(save_path)
    
# slow_tokenizer.save_pretrained(save_path)


In [ ]:
tokenizer = AlbertTokenizerFast.from_pretrained(save_path, do_lower_case=False, keep_accents=True) 

In [ ]:
#file_path = path + "json_dataset/Wiki_Dataset_Final.json"
file_path = path + "json_dataset/wiki_1050.json"
json_dataset = read_json(file_path)
#json_dataset["data"][144]["qas"][8]

In [ ]:
raw_dataset = json_to_list(json_dataset)
raw_dataset[0].class_print()

Number of questions:  6766
Question: Türkiye'nin topraklarının büyük bölümü nerededir?
Answer: Anadolu
Answer Start: 69
Answer End: 76
Context: Türkiye Cumhuriyeti ya da kısaca Türkiye, topraklarının büyük bölümü Anadolu'da, küçük bir bölümü ise Balkan Yarımadası'nın güneydoğu uzantısı olan Trakya'da yer alan ülke. Kuzeybatıda Bulgaristan, batıda Yunanistan, kuzeydoğuda Gürcistan, doğuda Ermenistan, İran ve Azerbaycan'ın ekslav toprağı Nahçıvan, güneydoğuda ise Irak ve Suriye komşusudur. Güneyini Kıbrıs adası ve Akdeniz. Batısını Ege Denizi ve kuzeyini Karadeniz çevreler. Marmara Denizi ise İstanbul Boğazı ve Çanakkale Boğazı ile birlikte Anadolu'yu Trakya'dan yani Asya'yı Avrupa'dan ayırır. Türkiye, Avrupa ve Asya'nın kavşak noktasında yer alması sayesinde önemli bir jeostratejik güce sahiptir.


In [ ]:
max_len = 384

dataset = []
for data in raw_dataset:
    flag = data.preprocess()
    if flag == 1:
      dataset.append(data)
    
print("Dataset len: ", len(dataset))

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors


Dataset len:  5087


In [ ]:
train, test = train_test_split(dataset)

In [ ]:
print(len(train))
print(len(test))

4579
508


In [ ]:
save_data_as_file(test, "test_384_albert")
save_data_as_file(train, "train_384_albert")

In [ ]:
max_len = 384
train = read_saved_data("train_" + str(max_len) + "_albert")
test = read_saved_data("test_" + str(max_len) + "_albert")

In [ ]:
x_train, y_train = create_input_targets(train)
x_test, y_test = create_input_targets(test)

print(len(x_train[0]), len(x_test[0]))

4579 508


In [ ]:
configuration = transformers.AlbertConfig()  # default parameters and configuration for ALBERT

In [ ]:
from tensorflow.python.distribute.cluster_resolver.tpu.tpu_cluster_resolver import is_running_in_gce  # pylint: disable=unused-import
from tensorflow.python.distribute.cluster_resolver.tpu.tpu_cluster_resolver import TPUClusterResolver
from tensorflow.python.util.tf_export import tf_export

In [ ]:
tpu_address=""
def check_tpu_statue():
    import os
    
    if 'COLAB_TPU_ADDR' not in os.environ:
      print('ERROR: Not connected to a TPU runtime')
    else:
      tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
      print ('TPU address is', tpu_address)

check_tpu_statue()
# output: TPU address is grpc://10.70.191.234:8470

TPU address is grpc://10.28.24.58:8470


In [ ]:

use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.28.24.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.24.58:8470


...

Downloading:   0%|          | 0.00/634 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.6M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFAlbertModel.

All the layers of TFAlbertModel were initialized from the model checkpoint at loodos/albert-base-turkish-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 albert (TFAlbertMainLayer)     TFBaseModelOutputWi  11939584    ['input_1[0][0]',                
                                thPooling(last_hidd               'input_3[0][0]',            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
print(x_train)

[array([[    2,   101, 18583, ...,     0,     0,     0],
       [    2,  3739,  1190, ...,     0,     0,     0],
       [    2,  2770, 31274, ...,     0,     0,     0],
       ...,
       [    2, 24269, 31257, ...,     0,     0,     0],
       [    2,  1350, 31274, ...,     0,     0,     0],
       [    2,  1729, 31259, ...,     0,     0,     0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])]


In [ ]:
# Load Weights from Drive
# model.load_weights(path + "models/albertV1_weights.h5")
BATCH_VALUE=128
EPOCH_VALUE=10
model.fit(
    x_train,
    y_train,
    epochs=EPOCH_VALUE, # use 3 or 5
    verbose=2,
    batch_size=BATCH_VALUE,
)

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


36/36 - 83s - loss: 5.8595 - activation_loss: 2.6875 - activation_1_loss: 3.1720 - 83s/epoch - 2s/step
Epoch 2/10
36/36 - 10s - loss: 3.0798 - activation_loss: 1.3582 - activation_1_loss: 1.7216 - 10s/epoch - 291ms/step
Epoch 3/10
36/36 - 10s - loss: 2.1989 - activation_loss: 0.9360 - activation_1_loss: 1.2629 - 10s/epoch - 291ms/step
Epoch 4/10
36/36 - 10s - loss: 1.6862 - activation_loss: 0.7112 - activation_1_loss: 0.9751 - 10s/epoch - 291ms/step
Epoch 5/10
36/36 - 11s - loss: 1.3730 - activation_loss: 0.5668 - activation_1_loss: 0.8062 - 11s/epoch - 292ms/step
Epoch 6/10
36/36 - 11s - loss: 1.0906 - activation_loss: 0.4555 - activation_1_loss: 0.6351 - 11s/epoch - 292ms/step
Epoch 7/10
36/36 - 10s - loss: 0.8653 - activation_loss: 0.3418 - activation_1_loss: 0.5235 - 10s/epoch - 291ms/step
Epoch 8/10
36/36 - 10s - loss: 0.7012 - activation_loss: 0.2898 - activation_1_loss: 0.4114 - 10s/epoch - 290ms/step
Epoch 9/10
36/36 - 10s - loss: 0.6252 - activation_loss: 0.2522 - activation_1

In [ ]:
print(len(x_test[0]))
print(len(x_train[0]))

508
4579


In [ ]:
pred_start, pred_end = model.predict(x_test)
count = 0
results = []
total_f1 = 0
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
  element = test[idx]
  offsets = element.context_token_to_char
  start = np.argmax(start)
  end = np.argmax(end)

  if start >= len(offsets):
    continue

  pred_char_start = offsets[start][0]

  if end < len(offsets):
    pred_char_end = offsets[end][1]
    pred_ans = element.context[pred_char_start:pred_char_end]
  else:
    pred_ans = element.context[pred_char_start:]

  pred_tokens = pred_ans.split()
  true_tokens = element.answer.split()
  common = collections.Counter(true_tokens) & collections.Counter(pred_tokens)
  num_same = sum(common.values())

  if len(true_tokens) == 0 or len(pred_tokens) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    f1 =  int(true_tokens == pred_tokens)
  elif num_same == 0:
    f1 =  0
  else:
    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(true_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
  total_f1 += f1

  results.append({
      "question": element.question,
      "true answer": element.answer,
      "predicted answer": pred_ans,
      "context": element.context,
      "f1 score": f1,
  })

  # print(f"Question: {element.question}")
  # print(f"Prediction: {pred_ans}\nTrue Answer: {element.answer}")
  # print(f"Context: {element.context}")
  # print("\n")

  #İlk eleman boşluk olduğu durumda kaldırıyoruz
  if(element.answer[0]==' '):
    element.answer = element.answer[1:]

  if(len(pred_ans)>1 and pred_ans[0]==' '):
    pred_ans = pred_ans[1:]


  if pred_ans == element.answer:
    count += 1

acc = count / len(y_test[0])
F1 = total_f1 / len(y_test[0])

# print(f"ALBERT MODEL: exact match:={acc:.2f} f1:={F1:.2f}")

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>]


In [ ]:
print("ALBERT MODEL\n Batch Size:"+ str(BATCH_VALUE)+"\n Epoch:+"+str(EPOCH_VALUE)+f"\n exact match:={acc:.2f} f1:={F1:.2f}")

ALBERT MODEL
 Batch Size:128
 Epoch:+10
 exact match:=0.57 f1:=0.73


In [ ]:
for item in results:
  print(item["question"])
  print(item["true answer"])
  print(item["predicted answer"])
  #print(item["context"])
  print(item["f1 score"])

Cannes Film Festivali'nde Altın Palmiye için her yıl kaç film yarışır?
ortalama 20 film
 20
0.5
Atomlar çok küçük olduğundan, bunların özellikleri nasıl anlaşılabilir?
dolaylı deney teknikleriyle
 dolaylı deney teknikleriyle
1.0
Jean Foyer ne eğitimi görmüştür?
Hukuk
 Hukuk
1.0
Akropolisler ne zaman sadece tapınaklara ayrılmışlardır?
MÖ 5. yüzyılda
 MÖ 5. yüzyılda
1.0
Jainistler hâlâ neyi devam ettirmektedirler?
antik Şraman'ı (श्रमण) - bir tür sofu (çileci) gelenek -
 antik Şraman'ı (श्रमण) - bir tür sofu (çileci) gelenek -
1.0
Pixar, Disney'e kaç dolara satılmıştır?
7,4 milyar
 7,4 milyar
1.0
Şıhlar Camii kaç yılında inşa edilmiştir?
1888
 1888
1.0
Beslenme, solunum, dolaşım, boşaltım, üreme gibi faaliyetlere ne ad verilir?
yaşamsal faaliyet
araştırma
0
Elektriğin Türkçe eş anlamlısı hangi sözcüktür?
çıngı
 çıngı
1.0
İlluminati neye karşı kurulmuştur?
batıl inanca, ön yargıya, dinin sosyal hayat üzerindeki etkisine, iktidarın kötüye kullanımına karşı

0
Saint Joseph Günü'ne İsviçre'n

In [ ]:
with open(save_path + "/results/" +"loodos-albert-base-turkish-uncased" + "_10epoch_result.txt", "w") as f:
  for result in results:
    f.write('%s\n' %result)

model.save_weights(save_path + "/weights/" + "loodos-albert-base-turkish-uncased" + "_seqlen512_batch64_epochs10_weights.h5")

In [ ]:
# save_model_name = splitted_model[0] + "-" + splitted_model[1]
# file_name = save_path + "test-results/" + save_model_name  + "_10epochs_results.txt"
# with open(file_name, "w") as f:
#   for result in results:
#     f.write('%s\n' %result)

In [ ]:
# model.save_weights(save_path + "weights/" + save_model_name + "_seqlen512_epochs12"".h5")

In [ ]:
print(save_path)

/content/gdrive/MyDrive/Q&A projesi/models/loodos-albert-base-turkish-uncased
